In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\
    
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from definitions import ROOT_DIR

In [2]:
import pandas as pd
import numpy as np

In [3]:
from src.features.multi_omics import MultiOmicsData

lusc_data = MultiOmicsData(cancer_type="LUSC", 
                           folder_path=ROOT_DIR+"/data/tcga-assembler/LUSC/", 
                           modalities=["GE", "MIR"])

/opt/anaconda3/envs/py27/lib/python2.7/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


('DRUGS', (357, 4))
('PATIENTS', (504, 5))
('MIR', (380, 1870))
('GE', (552, 20472))


In [4]:
centroids = pd.read_csv(ROOT_DIR+"/data/external/wilkerson.scc/predictor.centroids.csv")
centroids.columns = ["genes", "primitive", "classical", "secretory", "basal"]
centroids = centroids[centroids["genes"].isin(lusc_data.GE.get_genes_list())]
centroid_genes = centroids["genes"]

In [5]:
centroids.index = centroids.genes
centroids.drop(['genes'], axis=1, inplace=True)
centroids = centroids.T
centroids

genes,MYL6B,PODXL2,HSF2,TTLL4,MARCKSL1,MDK,CHKA,TRIM28,STOM,CASP1,...,ALDH1A3,DSE,MMP10,VDR,CAPZB,FNBP1,ENPP4,SH2B3,DOCK10,SDC1
primitive,0.539568,0.852272,0.293831,0.679557,1.015985,1.001421,0.513166,0.569403,-0.737997,-0.775559,...,-0.231312,-0.521502,-0.862938,-0.421701,-0.015213,-0.016889,0.232617,-0.178687,-0.121821,-0.764157
classical,-0.139755,-0.019997,0.060404,-0.089302,-0.306555,-0.019797,-0.150269,0.021242,-0.065057,-0.187048,...,-0.485903,-0.462036,-0.649346,-0.259443,-0.216407,-0.103185,-0.276022,-0.169656,-0.137791,0.337130
secretory,-0.084411,-0.104802,-0.131566,-0.044189,0.012595,-0.202898,0.235328,-0.264212,0.297794,0.305713,...,0.325191,0.487990,-0.974158,0.384095,0.180478,0.421356,0.482619,0.533466,0.569353,-0.998660
basal,-0.033551,-0.053003,-0.079130,-0.084657,0.097278,-0.222180,-0.136313,0.011287,0.043434,0.063549,...,0.905489,0.509466,2.926716,0.309637,0.216626,-0.185309,-0.147701,-0.030816,-0.093761,0.419894


# Subsetting the GE data to only genes

In [8]:
lusc_ge = lusc_data.GE.data[centroid_genes]


# Subset the LUSC samples to only tumor samples
lusc_ge = lusc_ge[lusc_ge.index.str.contains("-01A")]

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-np.max(np.abs(centroids.values)), np.max(np.abs(centroids.values))))
# scaler = MinMaxScaler()
# scaler.fit(centroids)

# Classify LUSC patients based on cluster centroids obtained from Wilkenson, et. al

TODO Read paper "The molecular portraits of breast tumors are conserved across microarray platforms" on how to do subtype prediction

In [9]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)

In [10]:
kmeans.cluster_centers_ = centroids.values

In [11]:
lusc_subtypes_map = {0: 'Primitive', 1: 'Classical', 2: 'Secretory', 3: 'Basal'}

lusc_subtypes_pred = pd.DataFrame(kmeans.predict(scaler.fit_transform(lusc_ge)), index=lusc_ge.index)
lusc_subtypes_pred.columns = ["subtype"]
lusc_subtypes_pred.replace({"subtype": lusc_subtypes_map}, inplace=True)
lusc_subtypes_pred["subtype"].value_counts(sort=False, normalize=False)

Secretory     40
Primitive     23
Classical    221
Basal        212
Name: subtype, dtype: int64

# Assign predicted subtypes to LUSC patients samples

In [12]:
lusc_subtypes_pred["patient_barcode"] = lusc_subtypes_pred.index.str[:-4]

lusc_data.add_subtypes_to_patients_clinical(dict(zip(lusc_subtypes_pred["patient_barcode"], lusc_subtypes_pred["subtype"])))

In [13]:
X, y = lusc_data.load_data(multi_omics=["GE"], 
#                            target=["ajcc_pathologic_tumor_stage"], 
                           target=["predicted_subtype"], 
                           predicted_subtypes=["Primitive"], 
#                            pathologic_stages=['Stage I', 'Stage II', 'Stage III', 'Stage IV']
                          )
print X["GE"].shape , y.shape

(29, 20472) (29, 1)


# Putative Associations

In [14]:
from src.data.make_dataset import Target_Scan
target_scan = Target_Scan(mirna_list=lusc_data.MIR.get_genes_list(), gene_symbols=lusc_data.GE.get_genes_list())

# Load miRanda putative miRNA-target associations
miRanda_df = pd.read_table(os.path.join(ROOT_DIR, 'data/external/miRanda_hg19_predictions_S_C_aug2010.txt'), delimiter='\t')
miRanda_df = miRanda_df[['mirna_name',  'gene_symbol']]

miRanda_df['mirna_name'] = miRanda_df['mirna_name'].str.lower()
miRanda_df['mirna_name'] = miRanda_df['mirna_name'].str.replace("*", "")
miRanda_df['mirna_name'] = miRanda_df['mirna_name'].str.replace("-3p.*|-5p.*", "")
print miRanda_df.shape

miRanda_df.columns = ["MiRBase ID", "Gene Symbol"]
miRanda_df = miRanda_df[miRanda_df['MiRBase ID'].isin(lusc_data.MIR.get_genes_list()) & miRanda_df['Gene Symbol'].isin(lusc_data.GE.get_genes_list())].dropna().drop_duplicates()
print miRanda_df.shape

# Union TargetScan and MiRanDa miRNA-target associations
targetScan_df = target_scan.get_miRNA_target_interaction_context()
putative_assocs = pd.concat([targetScan_df, miRanda_df]).dropna().drop_duplicates()
print putative_assocs.shape

(1097064, 2)
(532659, 2)
(603489, 2)


# Run MDSN dysregulation analyses

In [ ]:
from src.models.miRNA_target_network import miRNATargetNetwork as miRNATargetNet
import warnings
warnings.filterwarnings('ignore')

A, _ = lusc_data.load_data(multi_omics=["MIR", "GE"], predicted_subtypes=["Secretory"])
B, _ = lusc_data.load_data(multi_omics=["MIR", "GE"], predicted_subtypes=["Classical"])
network_subtype_sc = miRNATargetNet(miRNAs=lusc_data.MIR.get_genes_list(), targets=lusc_data.GE.get_genes_list())
print network_subtype_sc.fit(tag="Secretory-Classical", p_threshold=0.01, n_jobs=8,
            miRNA_A=A["MIR"],
            gene_A=A["GE"],
            miRNA_B=B["MIR"],
            gene_B=B["GE"],
            putative_assocs=putative_assocs)

A, _ = lusc_data.load_data(multi_omics=["MIR", "GE"], predicted_subtypes=["Classical"])
B, _ = lusc_data.load_data(multi_omics=["MIR", "GE"], predicted_subtypes=["Basal"])
network_subtype_cb = miRNATargetNet(miRNAs=lusc_data.MIR.get_genes_list(), targets=lusc_data.GE.get_genes_list())
print network_subtype_cb.fit(tag="Classical-Basal", p_threshold=0.01, n_jobs=8,
            miRNA_A=A["MIR"],
            gene_A=A["GE"],
            miRNA_B=B["MIR"],
            gene_B=B["GE"],
            putative_assocs=putative_assocs)

A, _ = lusc_data.load_data(multi_omics=["MIR", "GE"], predicted_subtypes=["Secretory"])
B, _ = lusc_data.load_data(multi_omics=["MIR", "GE"], predicted_subtypes=["Basal"])
network_subtype_sb = miRNATargetNet(miRNAs=lusc_data.MIR.get_genes_list(), targets=lusc_data.GE.get_genes_list())
print network_subtype_sb.fit(tag="Secretory-Basal", p_threshold=0.01, n_jobs=8,
            miRNA_A=A["MIR"],
            gene_A=A["GE"],
            miRNA_B=B["MIR"],
            gene_B=B["GE"],
            putative_assocs=putative_assocs)

n_A 28
n_B 154


In [ ]:
network_norm_tumor = miRNATargetNet(miRNAs=lusc_data.MIR.get_genes_list(), targets=lusc_data.GE.get_genes_list())
A, _ = lusc_data.load_data(multi_omics=["MIR", "GE"], pathologic_stages=["Normal"])
B, _ = lusc_data.load_data(multi_omics=["MIR", "GE"], pathologic_stages=["Stage I", "Stage II", "Stage III", "Stage IV"])
print network_norm_tumor.fit(tag="Secretory-Basal", p_threshold=0.01, n_jobs=8,
            miRNA_A=A["MIR"],
            gene_A=A["GE"],
            miRNA_B=B["MIR"],
            gene_B=B["GE"],
            putative_assocs=putative_assocs)

# Combine subtypes dysregulation networks into one

In [60]:
import networkx as nx

def relabel_mapping(node):
    # HACK: This function is used to relabel target nodes in the miRNA-target bipartite graph such that target nodes
    # in each distinct bipartite graphs have different names
    if "hsa-" in node:
        return node
    else:
        return node+"_"+str(np.random.randint(1000)) # a random number, so each run gives a "probably" different relabeling
    
nx.relabel_nodes(network_subtype_sc.B, relabel_mapping, copy=False)
nx.relabel_nodes(network_subtype_cb.B, relabel_mapping, copy=False)
nx.relabel_nodes(network_subtype_sb.B, relabel_mapping, copy=False)

In [62]:
network_subtypes = nx.compose_all([network_subtype_sc.B, network_subtype_cb.B, network_subtype_sb.B])

network_subtypes_combined = miRNATargetNet(miRNAs=lusc_data.MIR.get_genes_list(), targets=lusc_data.GE.get_genes_list())
network_subtypes_combined.B = network_subtypes

# sorted(network_subtypes.nodes())

In [70]:
print len(network_subtype_sc.B.edges())
print len(network_subtype_cb.B.edges())
print len(network_subtype_sb.B.edges())
print len(network_norm_tumor.B.edges())
print len(network_subtypes_combined.B.edges())

0
0
0
0
0


# Bipartite Graph Analysis

In [68]:
import networkx as nx
from networkx.algorithms import bipartite
import copy

# g = network_subtypes.to_undirected()
g = copy.deepcopy(network_subtypes_combined.B.to_undirected())
g.remove_nodes_from(list(nx.isolates(g)))


miRNAs_nodes = set(n for n, d in g.nodes(data=True) if d['bipartite'] == 0)
targets_nodes = set(g) - miRNAs_nodes
print 'mirnas', len(miRNAs_nodes)
print 'targets', len(targets_nodes)
print 'edges',len(g.edges())

mirnas 0
targets 0
edges 0
